In [1]:
from pymongo import MongoClient
import pymongo
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pickle
from tmqr.settings import *
from tmqrfeed.contracts import OptionContract

In [2]:
%load_ext Cython
%load_ext line_profiler
%load_ext memory_profiler

In [3]:
instrument = 'TST'
market = 'T'
future = 'F-TST-H11-110322'
opt_expiration = '110322'

In [4]:
options = ['C', 'P']
N = 60

In [5]:
strikes = list(np.arange(50.0, 100.0, 1.0))

In [6]:
options_tickers_list = []
for opt_type in options:
    for s in strikes:
        #US.C.F-CL-H11-110322.110121@89.0
        options_tickers_list.append("{0}.{1}.{2}.{3}@{4}".format(market, opt_type, future,  opt_expiration, s))        

In [7]:
dates = [datetime(2011, 1, 2) + timedelta(days=x) for x in range(N)]

client = MongoClient(MONGO_CONNSTR)
db = client[MONGO_DB]

In [8]:


for i, opt in enumerate(options_tickers_list):
    iv_arr = np.random.random(size=N)
    price_arr = np.random.random(size=N)
    to_exp = np.arange(N)
    
    df = pd.DataFrame({'iv': iv_arr, 'px': price_arr, 'toexp': to_exp}, index=dates)
    #
    # Writing bundled quotes
    # 
    db['quotes_options_eod_bundled'].create_index([('tckr', pymongo.ASCENDING)])
    db['quotes_options_eod_bundled'].create_index([('idx', pymongo.ASCENDING)])
    db['quotes_options_eod_bundled'].replace_one({'tckr': opt},
                                                 {'tckr': opt, 
                                                  'data': pickle.dumps(df),
                                                  'idx': i,
                                                 },
                                                 upsert=True
    )
    #
    # Writing single records
    # 
    db['quotes_options_eod_single'].create_index([('tckr', pymongo.ASCENDING), ('dt', pymongo.ASCENDING)])
    db['quotes_options_eod_single'].create_index([('idx', pymongo.ASCENDING)])
    for i, dt in enumerate(dates):
        """print({
            'tckr': opt,
            'dt': dt,
            'iv': iv_arr[i],
            'px': price_arr[i],
            'to_exp': to_exp[i]
        })
        """
        db['quotes_options_eod_single'].replace_one({'tckr': opt, 'dt': dt},
            {
                'tckr': opt,
                'dt': dt,
                'iv': iv_arr[i],
                'px': price_arr[i],
                'idx': i,
                #'to_exp': to_exp[i]
            }, upsert=True)
    
    
    
    

In [9]:
class ChainSingle:
    def __init__(self, opt_tickers):
        client = MongoClient(MONGO_CONNSTR)
        self.db = client[MONGO_DB]
        self.tickers = opt_tickers
        
    def compose(self, date):
        result = db['quotes_options_eod_single'].find({'tckr': {'$in': self.tickers}, 'dt': date})
        #result = db['quotes_options_eod_single'].find({'idx': {'$in': list(range(len(self.tickers)))}, 'dt': date})
        return pd.DataFrame(list(result))
    
    def get_one(self, tckr, date):
        result = db['quotes_options_eod_single'].find({'tckr': tckr, 'dt': date})
        return pd.DataFrame(list(result))
        
class ChainBundled:
    def __init__(self, opt_tickers):
        client = MongoClient(MONGO_CONNSTR)
        self.db = client[MONGO_DB]
        self.tickers = opt_tickers
        self.panel = None
        self.one_df = None
        
    def compose(self, date):
        if self.panel is None:
            result = db['quotes_options_eod_bundled'].find({'tckr': {'$in': self.tickers}})  
            #result = db['quotes_options_eod_bundled'].find({'idx': {'$in': list(range(len(self.tickers)))}})  
            
            res_dict = {}
            for res in result:
                res_dict[res['tckr']] = pickle.loads(res['data'])

            self.panel = pd.Panel(res_dict)
        return self.panel.major_xs(date)
    
    def get_one(self, tckr, date):
        if self.one_df is None:
            result = db['quotes_options_eod_bundled'].find_one({'tckr': tckr})        
        
            self.one_df = pickle.loads(result['data'])
        return self.one_df.loc[date]
        
        
        

# Chaing composition benchmark

In [10]:
#%%timeit 
chain1 = ChainSingle(options_tickers_list)
chain1.compose(datetime(2011, 1, 2))

,_id,dt,idx,iv,px,tckr
0,58d0ea006db8b05216969902,2011-01-02,0,0.426807,0.629637,T.C.F-TST-H11-110322.110322@50.0
1,58d0ea006db8b0521696993e,2011-01-02,0,0.884776,0.727405,T.C.F-TST-H11-110322.110322@51.0
2,58d0ea006db8b0521696997a,2011-01-02,0,0.385152,0.754213,T.C.F-TST-H11-110322.110322@52.0
3,58d0ea006db8b052169699b6,2011-01-02,0,0.319613,0.719323,T.C.F-TST-H11-110322.110322@53.0
4,58d0ea006db8b052169699f2,2011-01-02,0,0.494868,0.179635,T.C.F-TST-H11-110322.110322@54.0
5,58d0ea006db8b05216969a2e,2011-01-02,0,0.635267,0.240109,T.C.F-TST-H11-110322.110322@55.0
6,58d0ea006db8b05216969a6a,2011-01-02,0,0.815157,0.556721,T.C.F-TST-H11-110322.110322@56.0
7,58d0ea006db8b05216969aa6,2011-01-02,0,0.850256,0.425047,T.C.F-TST-H11-110322.110322@57.0
8,58d0ea006db8b05216969ae2,2011-01-02,0,0.563873,0.589014,T.C.F-TST-H11-110322.110322@58.0
9,58d0ea006db8b05216969b1e,2011-01-02,0,0.714433,0.221142,T.C.F-TST-H11-110322.110322@59.0


In [11]:
#%%timeit
chain2 = ChainBundled(options_tickers_list)
chain2.compose(datetime(2011, 1, 2))

,T.C.F-TST-H11-110322.110322@50.0,T.C.F-TST-H11-110322.110322@51.0,T.C.F-TST-H11-110322.110322@52.0,T.C.F-TST-H11-110322.110322@53.0,T.C.F-TST-H11-110322.110322@54.0,T.C.F-TST-H11-110322.110322@55.0,T.C.F-TST-H11-110322.110322@56.0,T.C.F-TST-H11-110322.110322@57.0,T.C.F-TST-H11-110322.110322@58.0,T.C.F-TST-H11-110322.110322@59.0,...,T.P.F-TST-H11-110322.110322@90.0,T.P.F-TST-H11-110322.110322@91.0,T.P.F-TST-H11-110322.110322@92.0,T.P.F-TST-H11-110322.110322@93.0,T.P.F-TST-H11-110322.110322@94.0,T.P.F-TST-H11-110322.110322@95.0,T.P.F-TST-H11-110322.110322@96.0,T.P.F-TST-H11-110322.110322@97.0,T.P.F-TST-H11-110322.110322@98.0,T.P.F-TST-H11-110322.110322@99.0
iv,0.426807,0.884776,0.385152,0.319613,0.494868,0.635267,0.815157,0.850256,0.563873,0.714433,...,0.226682,0.468639,0.270154,0.574432,0.207775,0.562074,0.012772,0.482977,0.738134,0.673959
px,0.629637,0.727405,0.754213,0.719323,0.179635,0.240109,0.556721,0.425047,0.589014,0.221142,...,0.629943,0.610076,0.979222,0.847582,0.980509,0.740181,0.494368,0.821358,0.500540,0.478498
toexp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
p = chain2.panel

In [33]:
mem = 0
for i in p:
    mem += p.loc[i].memory_usage()
print(mem)


Index    48000
iv       48000
px       48000
toexp    48000
dtype: int64


In [26]:
%%timeit
p.loc['T.C.F-TST-H11-110322.110322@50.0'].at[datetime(2011, 1, 2), 'iv']

The slowest run took 6.77 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 52.7 µs per loop


In [11]:
%timeit chain1.compose()

NameError: name 'chain1' is not defined

In [12]:
%timeit chain2.compose(datetime(2011, 1, 3))

NameError: name 'chain2' is not defined

In [ ]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
for i in range(20):
    chain1.compose(datetime(2011, 1, 2) + timedelta(days=i))

In [ ]:
%%timeit 
chain2 = ChainBundled(options_tickers_list)

for i in range(20):
    chain2.compose(datetime(2011, 1, 2) + timedelta(days=i))

# Single element fetching benchmark

In [ ]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
chain1.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2))

In [ ]:
%%timeit 
chain2 = ChainBundled(options_tickers_list)
chain2.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2))

In [ ]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
for i in range(20):
    chain1.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2) + timedelta(days=i))

In [ ]:
%%timeit 
chain2 = ChainBundled(options_tickers_list)

for i in range(20):
    chain2.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2) + timedelta(days=i))

In [ ]:
chain2 = ChainBundled(options_tickers_list)
chain2.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2))

# Option chains benchmarks

In [13]:
#%%timeit 
cursor = db['asset_index'].aggregate([
            {'$match': {
                'underlying': 'US.F.ES.H11.110318',
                'type': {'$in': ['P', 'C']},                
            }},
            
            {'$sort': {'strike': 1}},
            
            {'$project': {'tckr': 1, 'exp': 1, 'strike': 1, 'type': 1}
            },
            
            {'$group': {
                '_id': {'date': '$exp'},
                'chain': {'$push': '$$ROOT'},
            }
            },
            {'$sort': {"_id.date": 1}}          
])
chain_list = list(cursor)

In [14]:
%timeit [OptionContract(opt['tckr']) for opt in chain_list[0]['chain']]

100 loops, best of 3: 3.41 ms per loop


In [82]:
chain_list

[{'_id': {'date': datetime.datetime(2011, 1, 21, 0, 0)},
  'chain': [{'_id': ObjectId('58ad409933b18260b69de2c5'),
    'exp': datetime.datetime(2011, 1, 21, 0, 0),
    'strike': 375.0,
    'tckr': 'US.C.F-ES-H11-110318.110121@375.0',
    'type': 'C'},
   {'_id': ObjectId('58ad409a33b18260b69de6c2'),
    'exp': datetime.datetime(2011, 1, 21, 0, 0),
    'strike': 375.0,
    'tckr': 'US.P.F-ES-H11-110318.110121@375.0',
    'type': 'P'},
   {'_id': ObjectId('58ad409933b18260b69de2c6'),
    'exp': datetime.datetime(2011, 1, 21, 0, 0),
    'strike': 400.0,
    'tckr': 'US.C.F-ES-H11-110318.110121@400.0',
    'type': 'C'},
   {'_id': ObjectId('58ad409a33b18260b69de6c3'),
    'exp': datetime.datetime(2011, 1, 21, 0, 0),
    'strike': 400.0,
    'tckr': 'US.P.F-ES-H11-110318.110121@400.0',
    'type': 'P'},
   {'_id': ObjectId('58ad409933b18260b69de2c7'),
    'exp': datetime.datetime(2011, 1, 21, 0, 0),
    'strike': 425.0,
    'tckr': 'US.C.F-ES-H11-110318.110121@425.0',
    'type': 'C'},
   {

In [81]:
OptionContract(opt['tckr'])

TypeError: string indices must be integers

In [15]:
%lprun -f OptionContract.__init__ [OptionContract(opt['tckr']) for opt in chain_list[0]['chain']]

In [74]:
#%%timeit 
from collections import OrderedDict

chain_result = OrderedDict()

strike_count = 0

for exp in chain_list:
    options = chain_result.setdefault(exp['_id']['date'], OrderedDict())
    
    prev_strike = 0.0
    chain = exp['chain']
    for i, strike_rec in enumerate(chain):
        strike_count += 1
        strike = strike_rec['strike']
        if i == 0:
            continue
                
        if strike == chain[i-1]['strike']:
            # We have put call pair             
            if strike_rec['type'] == 'C':
                call_idx = i
                put_idx = i-1
            else:
                call_idx = i-1
                put_idx = i
            
            options[strike] = (chain[call_idx]['tckr'], chain[put_idx]['tckr'])    
    

In [75]:
strike_count

1110

In [73]:
with open("chain_list_es.pkl", 'wb') as f:
    pickle.dump(chain_list, f)

In [146]:
[x['_id']['date'] for x in chain_list]

[datetime.datetime(2011, 1, 21, 0, 0),
 datetime.datetime(2011, 2, 18, 0, 0),
 datetime.datetime(2011, 3, 18, 0, 0)]

In [25]:
%timeit chain2.panel.ix['T.C.F-TST-H11-110322.110322@50.0']

10000 loops, best of 3: 28.3 µs per loop


In [26]:
df = chain2.panel.ix['T.C.F-TST-H11-110322.110322@50.0']

In [29]:
%timeit df.at[datetime(2011,1, 2), 'iv']

The slowest run took 7.91 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 16.5 µs per loop


In [32]:
[x for x in [1, 2, 3]]

[1, 2, 3]